In [ ]:
######################################################################################### 
# Read secretes from Azure Key Vault
#########################################################################################
## This is the name of my Azure Key Vault 
key_vault = "https://INSERT_HERE.vault.azure.net/"
## I have stored my tenant id as one of the secrets to make it easier to use when needed 
tenant = mssparkutils.credentials.getSecret(key_vault , "INSERT_HERE") 
## This is my application Id for my service principal account 
client = mssparkutils.credentials.getSecret(key_vault , "INSERT_HERE") 
## This is my Client Secret for my service principal account 
client_secret = mssparkutils.credentials.getSecret(key_vault , "INSERT_HERE") 

######################################################################################### 
# Authentication - Replace string variables with your relevant values 
#########################################################################################  

import json, requests, pandas as pd 
import datetime  

try: 
    from azure.identity import ClientSecretCredential 
except Exception:
     !pip install azure.identity 
     from azure.identity import ClientSecretCredential 

# Generates the access token for the Service Principal 
api = 'https://analysis.windows.net/powerbi/api/.default' 
auth = ClientSecretCredential(authority = 'https://login.microsoftonline.com/', 
               tenant_id = tenant, 
               client_id = client, 
               client_secret = client_secret) 
access_token = auth.get_token(api)
access_token = access_token.token 

## This is where I store my header with the Access Token, because this is required when authenticating 
## to the Power BI Admin APIs 
header = {'Authorization': f'Bearer {access_token}'}  

print('\nSuccessfully authenticated.')

In [ ]:
# Welcome to your new notebook
# Type here in the cell editor to add code!
# Authenticate with Service Principal

# Welcome to your new notebook
# Type here in the cell editor to add code!
import msal
import requests
import json
import datetime
from datetime import datetime,date,timedelta

# Replace with your Azure AD app details
client_id = client
client_secret = client_secret
tenant_id = tenant

authority_url = f"https://login.microsoftonline.com/{tenant_id}"
graph_api_url = "https://graph.microsoft.com/v1.0"

# Create MSAL client application
app = msal.ConfidentialClientApplication(
    client_id, authority=authority_url, client_credential=client_secret
)

# Function to get an access token
def get_access_token():
    token_response = app.acquire_token_for_client(scopes=["https://graph.microsoft.com/.default"])
    return token_response.get("access_token")



In [ ]:
# Getting all Groups and Members with Pagination
# NOTE: This can take over 30 mins to get all the information.

import requests
import json
import datetime
from datetime import datetime,date,timedelta

# Replace with your access token
access_token = get_access_token()

def get_paginated_results(url, headers):
    results = []
    while url:
        response = requests.get(url, headers=headers)
        data = response.json()
        results.extend(data.get('value', []))
        url = data.get('@odata.nextLink')
    return results

def main():
    headers = {
        'Authorization': f'Bearer {access_token}'
    }

    # Get all groups
    groups_url = 'https://graph.microsoft.com/v1.0/groups'
    groups = get_paginated_results(groups_url, headers)

    all_groups_members = []

    # Get members of each group
    for group in groups:
        group_id = group['id']
        group_name = group['displayName'],
        # print(f"Fetching members for group: {group_name}")
        members_url = f'https://graph.microsoft.com/v1.0/groups/{group_id}/members'
        members = get_paginated_results(members_url, headers)
        all_groups_members.append({
            'group_name': group['displayName'],
            'group_id': group['id'],
            # 'members': [{'name': member['displayName'], 'id': member['id']} for member in members]
            'members':[{"user_id": member["id"], "user_displayName": member.get("displayName", "N/A"),"userprincipalName": member.get("userprincipalName", "N/A"),"Email": member.get("mail", "N/A")} for member in members]
        })

    # Save to file
    fileName_groups = 'groups_and_members_' + (datetime.today()).strftime('%Y%m%d') + '.json'

    # Write the output to a JSON file
    with open(f"/lakehouse/default/Files/Entra_Id/{fileName_groups}", "w") as json_file:
        json.dump(all_groups_members, json_file, indent=4)

if __name__ == '__main__':
    main()

In [ ]:
import json
import pandas as pd 
from pyspark.sql.functions import col, year, month, quarter, to_date, dayofmonth
import datetime
from datetime import datetime,date,timedelta
#from pandas.io.json import json_normalize 

with open(f'/lakehouse/default/Files/Entra_Id/groups_and_members_' + (datetime.today()).strftime('%Y%m%d') + '.json') as f:
#with open(df, encoding="utf-16") as f:
   data = json.load(f)

# Get the Expanded Members, by using the json normalize and then have the higher levels of the json structure.

df = pd.json_normalize(data, "members",['group_id','group_name'])

# # # Display the DataFrame
sparkDF=spark.createDataFrame(df)
# display(sparkDF)

# # Write to Table
sparkDF.write.mode("overwrite").format("delta").option("overwriteSchema", "true").saveAsTable('staging_tb_EntraID_Groups')

In [ ]:
# Insert or Update New Rows
# Source query to create a temporary view
groups_source_query = f"""
SELECT user_id,
			user_displayName,
			userprincipalName,
			Email,
			group_id,
			group_name
FROM staging_tb_entraid_groups
where Email is not null
"""

# Create a temporary view from the source query
spark.sql(groups_source_query).createOrReplaceTempView("groups_updates")

# Merge operation
merge_query = f"""
MERGE INTO tb_EntraID_Groups AS dim
USING groups_updates AS updates
    ON dim.group_id = updates.group_id
    and dim.user_displayName = updates.user_displayName
    and dim.userprincipalName = updates.userprincipalName
    and dim.Email = updates.Email
    and dim.user_id = updates.user_id
    and dim.group_name = updates.group_name

WHEN MATCHED THEN
  UPDATE SET
    dim.user_id = updates.user_id,
    dim.user_displayName = updates.user_displayName,
    dim.userprincipalName = updates.userprincipalName,
    dim.Email = updates.Email,
    dim.group_id = updates.group_id,
    dim.group_name = updates.group_name

WHEN NOT MATCHED THEN
  INSERT (
    
    user_id,
    user_displayName,
    userprincipalName,
    Email,
    group_id,
    group_name

  ) VALUES (

    updates.user_id,
    updates.user_displayName,
    updates.userprincipalName,
    updates.Email,
    updates.group_id,
    updates.group_name

  )
"""

# Execute the merge query
# Execute the merge query and get the result as a DataFrame
result_df = spark.sql(merge_query)

# Collect the result to a local variable
result = result_df.collect()[0]

# Extract the different values
num_affected_rows = result['num_affected_rows']
num_updated_rows = result['num_updated_rows']
num_deleted_rows = result['num_deleted_rows']
num_inserted_rows = result['num_inserted_rows']

# Print the values
print(f"Number of affected rows: {num_affected_rows}")
print(f"Number of updated rows: {num_updated_rows}")
print(f"Number of deleted rows: {num_deleted_rows}")
print(f"Number of inserted rows: {num_inserted_rows}")

In [ ]:
# # Get all Users and download into JSON File
# # If needed to get more details here is how: https://stackoverflow.com/questions/48229949/get-all-user-properties-from-microsoft-graph
# # Here is an example for the API to get all details: https://graph.microsoft.com/v1.0/groups?$filter=displayname eq 'sec_Office365_FabricAdmin_Role'&$expand=members

import requests
import msal
import datetime
from datetime import datetime,date,timedelta

# Microsoft Graph API endpoint for listing users
GRAPH_API_URL = 'https://graph.microsoft.com/v1.0/users?$select=id,displayName,mail,userPrincipalName,jobTitle,accountEnabled,createdDateTime,lastPasswordChangeDateTime,mailNickName'

# Get Access Token

access_token = get_access_token()

# Define headers for API request
headers = {
    'Authorization': 'Bearer ' + access_token,
    'Content-Type': 'application/json'
}

# Make the API request to list users
response = requests.get(GRAPH_API_URL, headers=headers)

# To hold all users
all_users = []

# Pagination loop
while GRAPH_API_URL:
    response = requests.get(GRAPH_API_URL, headers=headers)
    data = response.json()
    
    # Process users
    all_users.extend(data['value'])
    
    # Get the next page URL
    GRAPH_API_URL = data.get('@odata.nextLink')

# Create Directory if it does not exist
mssparkutils.fs.mkdirs("Files/Entra_Id/")

fileName = 'users_' + (datetime.today()).strftime('%Y%m%d') + '.json'

# Write the output to a JSON file
with open(f"/lakehouse/default/Files/Entra_Id/{fileName}", "w") as json_file:
    json.dump(all_users, json_file, indent=4)



In [ ]:
# # Getting the JSON Users File into Dataframe

import json
import pandas as pd 
from pyspark.sql.functions import col, year, month, quarter, to_date, dayofmonth
import datetime
from datetime import datetime,date,timedelta
#from pandas.io.json import json_normalize 

fileName = 'users_' + (datetime.today()).strftime('%Y%m%d') + '.json'

with open(f'/lakehouse/default/Files/Entra_Id/{fileName}') as f:
#with open(df, encoding="utf-16") as f:
   data = json.load(f)

# Get the Expanded Members, by using the json normalize and then have the higher levels of the json structure.
df = pd.json_normalize(data)

# # # Display the DataFrame
sparkDF=spark.createDataFrame(df)
sparkDF.head(3)

# Rename column 'onPremisesExtensionAttributes_extensionAttribute3' to 'PositionCode'
# sparkDF = sparkDF.withColumnRenamed('onPremisesExtensionAttributes.extensionAttribute1', 'extensionAttribute1')
# sparkDF = sparkDF.withColumnRenamed('onPremisesExtensionAttributes.extensionAttribute2', 'extensionAttribute2')
# sparkDF = sparkDF.withColumnRenamed('onPremisesExtensionAttributes.extensionAttribute3', 'PositionCode')
# sparkDF = sparkDF.withColumnRenamed('onPremisesExtensionAttributes.extensionAttribute4', 'extensionAttribute4')
# sparkDF = sparkDF.withColumnRenamed('onPremisesExtensionAttributes.extensionAttribute5', 'extensionAttribute5')
# sparkDF = sparkDF.withColumnRenamed('onPremisesExtensionAttributes.extensionAttribute6', 'extensionAttribute6')
# sparkDF = sparkDF.withColumnRenamed('onPremisesExtensionAttributes.extensionAttribute7', 'extensionAttribute7')
# sparkDF = sparkDF.withColumnRenamed('onPremisesExtensionAttributes.extensionAttribute8', 'extensionAttribute8')
# sparkDF = sparkDF.withColumnRenamed('onPremisesExtensionAttributes.extensionAttribute9', 'extensionAttribute9')
# sparkDF = sparkDF.withColumnRenamed('onPremisesExtensionAttributes.extensionAttribute10', 'extensionAttribute10')
# sparkDF = sparkDF.withColumnRenamed('onPremisesExtensionAttributes.extensionAttribute11', 'extensionAttribute11')
# sparkDF = sparkDF.withColumnRenamed('onPremisesExtensionAttributes.extensionAttribute12', 'extensionAttribute12')
# sparkDF = sparkDF.withColumnRenamed('onPremisesExtensionAttributes.extensionAttribute13', 'extensionAttribute13')
# sparkDF = sparkDF.withColumnRenamed('onPremisesExtensionAttributes.extensionAttribute14', 'extensionAttribute14')
# sparkDF = sparkDF.withColumnRenamed('onPremisesExtensionAttributes.extensionAttribute15', 'extensionAttribute15')

# Write Users to Lakehouse Table
sparkDF.write.mode("overwrite").format("delta").option("overwriteSchema", "true").saveAsTable('staging_tb_EntraID_Users')

In [ ]:
# Insert or Update New Rows for Users
# Source query to create a temporary view
users_source_query = f"""
SELECT id
      ,displayName
      ,mail
      
    
      
      
      
      ,createdDateTime
      ,lastPasswordChangeDateTime
      ,mailNickname
      ,userPrincipalName
  FROM staging_tb_entraid_users
"""

# Create a temporary view from the source query
spark.sql(users_source_query).createOrReplaceTempView("users_updates")

# Merge operation
merge_query = f"""
MERGE INTO tb_EntraID_Users AS dim
USING users_updates AS updates

    ON dim.id = updates.id
    

WHEN MATCHED THEN
  UPDATE SET

    dim.id = updates.id,
    dim.displayName = updates.displayName,
    dim.mail = updates.mail,
    
    
    
    
    
    dim.createdDateTime = updates.createdDateTime,
    dim.lastPasswordChangeDateTime = updates.lastPasswordChangeDateTime,
    dim.userPrincipalName = updates.userPrincipalName,
    
    dim.mailNickname = updates.mailNickname

WHEN NOT MATCHED THEN
  INSERT (
    
    id,
    displayName,
    mail,
    
    createdDateTime,
    
    lastPasswordChangeDateTime,
   
    userPrincipalName,
    
    mailNickname

  ) VALUES (

    updates.id,
    updates.displayName,
    updates.mail,
    
    updates.createdDateTime,
   
    updates.lastPasswordChangeDateTime,
    
    updates.userPrincipalName,
   
    updates.mailNickname


  )
"""

# Execute the merge query
# Execute the merge query and get the result as a DataFrame
result_df = spark.sql(merge_query)

# Collect the result to a local variable
result = result_df.collect()[0]

# Extract the different values
num_affected_rows = result['num_affected_rows']
num_updated_rows = result['num_updated_rows']
num_deleted_rows = result['num_deleted_rows']
num_inserted_rows = result['num_inserted_rows']

# Print the values
print(f"Number of affected rows: {num_affected_rows}")
print(f"Number of updated rows: {num_updated_rows}")
print(f"Number of deleted rows: {num_deleted_rows}")
print(f"Number of inserted rows: {num_inserted_rows}")

In [ ]:
################################
## Delete Staging Table data
################################

delete_upnStaging_source_query = f"""
DELETE 
FROM tb_staging_Entra_ID_User_Licenses

"""
#Run SQL Query
spark.sql(delete_upnStaging_source_query)

In [ ]:

############################################################
# # Get all Users Licenses into JSON FILE
############################################################

import requests
import msal
import datetime
from datetime import datetime,date,timedelta
import datetime
from pyspark.sql.types import IntegerType,BooleanType,DateType
from pyspark.sql.functions import col, year, month, quarter
from pyspark.sql import SparkSession
from pyspark.sql.functions import *


# Get the UserPrincipalName or UPN for all off the active Users

# Source query to create a temporary view
userprincipalname_source_query = f"""
SELECT 
			userPrincipalName
FROM tb_entraid_users

"""

#Create the Dataframe for the UPN
userprincipalname_result_df = spark.sql(userprincipalname_source_query)

# # Create a Spark session
spark = SparkSession.builder.appName("AddColumnExample").getOrCreate()

# Get the dataframe I want to loop through.
# I used the method "collect()" to use it as my looping function.
data_collect = userprincipalname_result_df.collect()

# looping thorough each row of the dataframe
for row in data_collect:
    
    # This is where I am assigning a variable name for each row in my looping dataset.
    var_upn = row["userPrincipalName"]

    display(var_upn)

    try:

        # Microsoft Graph API endpoint for listing users
        GRAPH_API_URL = f"https://graph.microsoft.com/v1.0/users/{var_upn}/licenseDetails?$select=skuPartNumber" 

        # Get Access Token

        access_token = get_access_token()

        # Define headers for API request
        headers = {
            'Authorization': 'Bearer ' + access_token,
            'Content-Type': 'application/json'
        }

        # Make the API request to list users
        response = requests.get(GRAPH_API_URL, headers=headers)

        # Get the Response from the API
        data = response.json()   

        # # To hold all users
        all_upn_licenses = []

        # Adding Additional Rows
        all_upn_licenses.extend(data['value'])

        #Create Pandas Data Frame
        df_license_users = pd.DataFrame(all_upn_licenses)

        # #Create SparkDataFrame
        spark_license_users = spark.createDataFrame(df_license_users)

        # Add Column with UserPrincipalName
        spark_license_users = spark_license_users.withColumn("userPrincipalName" , lit(var_upn))
        spark_license_users = spark_license_users.withColumn("Inserted_DateTime", lit(current_timestamp()))


        # Write Data to Table
        spark_license_users.write.mode("overwrite").option("mergeSchema", "true").format("delta").save("Tables/tb_staging_Entra_ID_User_Licenses")

    except Exception as e:
        Error: {e}  


In [ ]:
############################################################
# # Insert or Update Users with Licenses
############################################################

# Source query to create a temporary view
license_users_source_query = f"""
SELECT skuPartNumber
      ,userPrincipalName
      ,CAST(Inserted_DateTime as DATE) as Inserted_DateTime
  FROM tb_staging_Entra_ID_User_Licenses
"""

# Create a temporary view from the source query
spark.sql(license_users_source_query).createOrReplaceTempView("license_users_updates")

# Merge operation
license_users_merge_query = f"""
MERGE INTO tb_Entra_ID_User_Licenses AS dim
USING license_users_updates AS updates

    ON dim.userPrincipalName = updates.userPrincipalName 
    AND dim.skuPartNumber = updates.skuPartNumber    

WHEN MATCHED THEN
  UPDATE SET

    dim.userPrincipalName = updates.userPrincipalName,
    dim.Inserted_DateTime = updates.Inserted_DateTime,
    dim.skuPartNumber = updates.skuPartNumber

WHEN NOT MATCHED THEN
  INSERT (
    
    userPrincipalName,
    Inserted_DateTime,
    skuPartNumber

  ) VALUES (

    updates.userPrincipalName,
    updates.Inserted_DateTime,
    updates.skuPartNumber
  )
"""

# Execute the merge query
# Execute the merge query and get the result as a DataFrame
license_users_result_df = spark.sql(license_users_merge_query)

# Collect the result to a local variable
license_users_result = license_users_result_df.collect()[0]

# Extract the different values
num_affected_rows = license_users_result['num_affected_rows']
num_updated_rows = license_users_result['num_updated_rows']
num_deleted_rows = license_users_result['num_deleted_rows']
num_inserted_rows = license_users_result['num_inserted_rows']

# Print the values
print(f"Number of affected rows: {num_affected_rows}")
print(f"Number of updated rows: {num_updated_rows}")
print(f"Number of deleted rows: {num_deleted_rows}")
print(f"Number of inserted rows: {num_inserted_rows}")